In [1]:
# ===================================================================
#                                                                   #
#           TITLE:      CDS HW6                                     #
#           AUTHOR:     SCHINDLER, VICARIA, VIRGUEZ                 #
#           DUE DATE:   NOV 21, 2021                                #
#                                                                   #
# ===================================================================

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!pip install "drive/MyDrive/CDS/HW6/ourpythonlibrarywithclasses/dist/ourpythonlibwithclasses-0.1.0-py3-none-any.whl" --force-reinstall

Processing ./drive/MyDrive/CDS/HW6/ourpythonlibrarywithclasses/dist/ourpythonlibwithclasses-0.1.0-py3-none-any.whl
  Using cached scikit_learn-1.0.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (23.2 MB)
  Using cached pandas-1.3.3-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.3 MB)
  Using cached numpy-1.21.4-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached pytz-2021.3-py2.py3-none-any.whl (503 kB)
  Using cached scipy-1.7.2-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (38.2 MB)
  Using cached threadpoolctl-3.0.0-py3-none-any.whl (14 kB)
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: numpy
    Found existing install

In [2]:
import ourpythonlibwithclasses

In [3]:
# ===================================================================
#                       1) LOAD

from ourpythonlibwithclasses.loading import loading_functions

path = "drive/MyDrive/CDS/HW6/sample_diabetes_mellitus_data.csv"
data = loading_functions.Load_Split(path)

# Inspect
print(data.df.shape)
print(data.train.shape)
print(data.test.shape)

(10000, 53)
(5000, 53)
(5000, 53)


In [4]:
# ===================================================================
#                       2) PREPROCESS

from ourpythonlibwithclasses.preprocessing import preprocessing_functions

# Define rows and cols to be dropped NA and filled NA, respectively.        
rows = ['age', 'gender', 'ethnicity']
cols  = ['height', 'weight']

# Drop rows w. NA 

# Training data
df_train_p = preprocessing_functions.PreProcessor(data.train)
df_train_p.clean_rows(rows)
df_train_p = df_train_p.cleaned
print(df_train_p.shape)

# Test data
df_test_p = preprocessing_functions.PreProcessor(data.test)
df_test_p.clean_rows(rows)
df_test_p = df_test_p.cleaned
print(df_test_p.shape)
df_test_p.head()

(4682, 53)
(4686, 53)


,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
1443,1443,198415,118,65.0,21.680499,1,Caucasian,F,167.6,Operating Room,Operating Room / Recovery,97,admit,MICU,3.850000,0,60.90,NaN,202.0,1203.02,1,0,NaN,6.0,0.48,NaN,4.0,6.0,0.0,5.0,77.0,108.0,28.1,0,118.0,NaN,NaN,NaN,NaN,46.0,137.0,NaN,NaN,0,5.4,0,0,0,0,0,0,0,0
3104,3104,253376,81,65.0,26.615539,0,Caucasian,M,170.2,Emergency Department,Accident & Emergency,90,admit,Med-Surg ICU,0.001389,0,77.10,NaN,113.0,501.05,0,0,NaN,18.0,1.12,NaN,4.0,6.0,0.0,5.0,222.0,120.0,22.8,0,56.0,NaN,NaN,NaN,NaN,39.0,139.0,36.4,NaN,1,3.0,0,0,0,0,0,0,1,0
6144,6144,218508,118,25.0,NaN,0,Caucasian,M,165.1,Emergency Department,Accident & Emergency,114,admit,CCU-CTICU,0.003472,0,NaN,NaN,113.0,501.05,0,0,NaN,12.0,1.11,NaN,4.0,6.0,0.0,5.0,165.0,124.0,36.7,0,66.0,NaN,NaN,NaN,NaN,36.0,137.0,37.4,NaN,1,7.6,0,0,0,0,0,0,0,0
7929,7929,221704,149,54.0,23.050656,0,Caucasian,M,182.9,Emergency Department,Accident & Emergency,103,admit,SICU,0.023611,0,77.11,NaN,119.0,601.03,0,0,NaN,NaN,NaN,NaN,3.0,6.0,0.0,4.0,NaN,107.0,NaN,0,111.0,NaN,NaN,NaN,NaN,40.0,NaN,35.4,NaN,0,NaN,0,0,0,0,0,0,0,0
5777,5777,214291,77,36.0,32.001713,0,Caucasian,M,185.4,Direct Admit,Floor,113,readmit,Med-Surg ICU,1.000000,0,110.00,NaN,301.0,403.01,0,0,NaN,8.0,1.00,NaN,2.0,5.0,0.0,1.0,77.0,43.0,42.8,0,133.0,NaN,NaN,NaN,NaN,16.0,140.0,36.1,NaN,0,6.2,0,0,0,0,0,0,0,0


In [5]:
# Fill cols w. NA (w. dropped rows w. NA)

# Training data
train_clean = preprocessing_functions.PreProcessor(df_train_p)   
train_clean.fill_na_mean(cols)
df_train_pc = train_clean.filled
print(df_train_pc[cols].isnull().sum())

# Test data
test_clean = preprocessing_functions.PreProcessor(df_test_p)   
test_clean.fill_na_mean(cols)
df_test_pc = test_clean.filled
print(df_test_pc[cols].isnull().sum())

height    0
weight    0
dtype: int64
height    0
weight    0
dtype: int64


/usr/local/lib/python3.7/dist-packages/ourpythonlibwithclasses/preprocessing/preprocessing_functions.py:18: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  self.filled = self.df[cols].fillna(self.df.mean())


In [6]:
# Tranform some features

# List of numeric features to transform (without our target variable)
numeric_features =  list(df_train_pc.select_dtypes('number').columns)
numeric_features.remove('diabetes_mellitus')

# Standarization of the numeric features

# Training data
df_train_pcs = df_train_pc.copy()
df_train_pcs[numeric_features] = preprocessing_functions.ReScaling('standarization').transform(df_train_pcs[numeric_features].copy())
print(df_train_pcs.shape)

# Test data
df_test_pcs = df_test_pc.copy()
df_test_pcs[numeric_features] = preprocessing_functions.ReScaling('standarization').transform(df_test_pcs[numeric_features].copy())
print(df_test_pcs.shape)

df_test_pcs.head()

(4682, 53)
(4686, 53)


,height,weight,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
1443,-0.233501,-1.115465e+00,-1.234279,-0.381438,0.476285,0.152715,-0.996354,1.827068,Caucasian,F,Operating Room,Operating Room / Recovery,-0.488128,admit,MICU,1.562827,NaN,NaN,0.176272,1.215069,1.748282,-0.162789,NaN,-0.913710,-0.707007,NaN,0.458431,0.347887,-0.025392,0.493856,-0.929178,0.308795,-0.760392,-0.452994,0.534283,NaN,NaN,NaN,NaN,0.923165,-0.198352,NaN,NaN,-0.664311,-0.908954,-0.020661,-0.138328,-0.13009,-0.227686,-0.091601,-0.073229,-0.162789,0
3104,0.011192,-4.176465e-01,-0.653445,1.064892,-0.725795,0.152715,-0.400321,-0.547208,Caucasian,M,Emergency Department,Accident & Emergency,-0.900585,admit,Med-Surg ICU,-0.303286,NaN,NaN,-0.875756,-0.218357,-0.571868,-0.162789,NaN,-0.348788,-0.283102,NaN,0.458431,0.347887,-0.025392,0.493856,0.753227,0.694288,-1.547119,-0.452994,-0.981076,NaN,NaN,NaN,NaN,0.477341,0.204945,-0.047961,NaN,1.504999,-1.259309,-0.020661,-0.138328,-0.13009,-0.227686,-0.091601,-0.073229,6.141598,0
6144,-0.468784,-6.488633e-14,0.409611,0.147321,0.476285,-2.275178,NaN,-0.547208,Caucasian,M,Emergency Department,Accident & Emergency,0.513553,admit,CCU-CTICU,-0.302275,NaN,NaN,-0.875756,-0.218357,-0.571868,-0.162789,NaN,-0.631249,-0.289726,NaN,0.458431,0.347887,-0.025392,0.493856,0.091868,0.822785,0.516182,-0.452994,-0.736663,NaN,NaN,NaN,NaN,0.286274,-0.198352,1.160423,NaN,1.504999,-0.587795,-0.020661,-0.138328,-0.13009,-0.227686,-0.091601,-0.073229,-0.162789,0
7929,1.206427,-4.172158e-01,1.033806,0.231426,1.483433,-0.514956,-0.830873,-0.547208,Caucasian,M,Emergency Department,Accident & Emergency,-0.134594,admit,SICU,-0.292511,NaN,NaN,-0.804832,-0.014198,-0.571868,-0.162789,NaN,NaN,NaN,NaN,-0.691850,0.347887,-0.025392,-0.279165,NaN,0.276670,NaN,-0.452994,0.363194,NaN,NaN,NaN,NaN,0.541030,NaN,-1.256345,NaN,-0.664311,NaN,-0.020661,-0.138328,-0.13009,-0.227686,-0.091601,-0.073229,-0.162789,0
5777,1.441710,9.995271e-01,0.281275,0.036348,-0.855749,-1.607508,0.250198,-0.547208,Caucasian,M,Direct Admit,Floor,0.454631,readmit,Med-Surg ICU,0.180920,NaN,NaN,1.346504,-0.418555,-0.571868,-0.162789,NaN,-0.819556,-0.362584,NaN,-1.842130,-0.521270,-0.025392,-2.598229,-0.929178,-1.779292,1.421659,-0.452994,0.900902,NaN,NaN,NaN,NaN,-0.987509,0.406593,-0.410476,NaN,-0.664311,-0.792169,-0.020661,-0.138328,-0.13009,-0.227686,-0.091601,-0.073229,-0.162789,0


In [7]:
# Polinomial features of some numeric variables (only 3: column **2 and column **3)

poly_columns = ['age', 'bmi']

# Training data
df_train_pcsp = df_train_pcs.copy()
df_train_pcsp = preprocessing_functions.PolyFeatures(n_poly_f = 3).transform(df_train_pcsp.copy(), poly_columns)
print(df_train_pcsp.shape)

# Test data
df_test_pcsp = df_test_pcs.copy()
df_test_pcsp = preprocessing_functions.PolyFeatures(n_poly_f = 3).transform(df_test_pcsp.copy(), poly_columns)
print(df_test_pcsp.shape)

df_train_pcsp.head()

(4682, 57)
(4686, 57)


,height,weight,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus,age_poly2,age_poly3,bmi_poly2,bmi_poly3
9247,-1.682367,-3.627257e-01,1.492997,-1.137092,-0.474903,1.240896,0.435172,-0.553440,Caucasian,M,Direct Admit,Accident & Emergency,1.644694,admit,Med-Surg ICU,-0.271683,NaN,NaN,-0.884882,-0.216780,-0.576795,-0.171631,NaN,NaN,NaN,0.146718,0.440855,-0.532609,-0.014662,0.481232,-1.167864,-0.089134,NaN,-0.458011,-1.038348,2.773706,2.773706,-0.558434,-1.582897,0.855072,NaN,-0.303250,-1.171702,1.527828,NaN,-0.02067,-0.139188,-0.110013,-0.220963,-0.086776,-0.076151,-0.168976,0,1.539822,1.910759,0.189375,0.082411
5483,0.955565,-1.263901e-14,0.185738,1.182153,0.475029,0.461417,NaN,1.806494,Caucasian,M,Operating Room,Operating Room / Recovery,-0.765980,admit,CTICU,-0.283388,NaN,NaN,1.340268,1.225727,1.733347,-0.171631,NaN,NaN,NaN,0.146718,0.440855,0.345818,-0.014662,0.481232,0.158362,0.423087,-0.047914,2.182886,-1.013354,0.563460,0.563460,-0.216366,-0.790353,0.405921,NaN,-0.388078,NaN,1.527828,0.357566,-0.02067,-0.139188,-0.110013,-0.220963,-0.086776,-0.076151,-0.168976,1,0.212906,0.098238,NaN,NaN
7747,1.201394,-1.120443e-02,0.972038,1.197544,0.475029,0.641297,-0.480051,-0.553440,African American,M,Emergency Department,Accident & Emergency,-0.354402,admit,Med-Surg ICU,-0.281232,NaN,-0.580815,-0.884882,-0.216780,-0.576795,-0.171631,0.141327,0.441808,-0.366147,NaN,0.440855,0.345818,-0.014662,0.481232,0.560966,1.959752,-2.338687,-0.458011,-1.238299,NaN,NaN,NaN,NaN,1.817538,1.156177,0.460202,NaN,-0.654384,2.110044,-0.02067,-0.139188,-0.110013,-0.220963,-0.086776,-0.076151,-0.168976,0,0.411262,0.263741,0.230449,-0.110628
2234,-1.682367,1.623581e+00,-0.942658,-1.221039,0.475029,0.401457,2.793007,1.806494,Caucasian,F,Operating Room,Operating Room / Recovery,-1.177559,admit,SICU,-0.279384,NaN,-0.142281,1.375587,2.235625,1.733347,-0.171631,-0.202322,-0.649502,-0.372833,-0.659358,0.440855,0.345818,-0.014662,0.481232,1.046460,-1.529757,0.543253,2.182886,1.610999,-0.091428,-0.091428,0.016240,-0.111030,0.405921,0.012152,-0.024530,NaN,1.527828,-0.256550,-0.02067,-0.139188,-0.110013,-0.220963,-0.086776,-0.076151,-0.168976,0,0.161168,0.064702,7.800888,21.787936
1405,0.482817,-7.896757e-02,-1.230574,0.207811,-0.671441,-1.037580,-0.272758,1.806494,Caucasian,F,Operating Room,Operating Room / Recovery,-0.589589,admit,Med-Surg ICU,-0.277536,NaN,NaN,0.162940,1.219524,1.733347,-0.171631,NaN,-0.897527,-0.539988,-1.143003,0.440855,0.345818,-0.014662,0.481232,0.359664,0.038921,0.410240,-0.458011,0.911171,0.972764,0.972764,-0.558434,-0.903574,1.496716,0.202823,0.217836,NaN,-0.654384,1.421036,-0.02067,-0.139188,-0.110013,-0.220963,-0.086776,-0.076151,-0.168976,0,1.076573,-1.117031,0.074397,-0.020292


In [8]:
# Before training we need to do some more preprocessing

df_train_pcsp.fillna(df_train_pcsp.mean(), inplace=True)
df_train_pcsp = df_train_pcsp.select_dtypes('number')
df_train_pcsp.drop(['readmission_status', 'gcs_unable_apache'], axis=1, inplace=True)
print(df_train_pcsp.shape)


df_test_pcsp.fillna(df_test_pcsp.mean(), inplace=True)
df_test_pcsp = df_test_pcsp.select_dtypes('number')
df_test_pcsp.drop(['readmission_status', 'gcs_unable_apache'], axis=1, inplace=True)
print(df_test_pcsp.shape)

(4682, 49)
(4686, 49)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  if __name__ == '__main__':


In [9]:
# ===================================================================
#                       3) MODELLING
from ourpythonlibwithclasses.modelling import modelling_functions

# Training the model
model = modelling_functions.Model(df_train_pcsp.drop('diabetes_mellitus', axis = 1), df_train_pcsp['diabetes_mellitus'])
model.train(alpha=1, regularization = "l2")

In [10]:
y_hat_proba_train, y_hat_train = model.predict(df_train_pcsp.drop('diabetes_mellitus', axis = 1))
print('AUC Training data')
modelling_functions.asses(df_train_pcsp['diabetes_mellitus'], y_hat_train)

AUC Training data
0.6624582627242154


In [11]:
y_hat_proba_test, y_hat_test = model.predict(df_test_pcsp.drop('diabetes_mellitus', axis = 1))
print('AUC Test data')
modelling_functions.asses(df_test_pcsp['diabetes_mellitus'], y_hat_test)

AUC Test data
0.6576900699214285
